In [0]:
import json
from glob import glob
from functions.sanity import validate_settings, initialize_empty_tables
from functions.job import save_job_configuration
from functions.utility import apply_job_type
from functions.project_root import PROJECT_ROOT

catalog = dbutils.widgets.get('catalog')
# Save the project root folder for other tasks to use
project_root = str(PROJECT_ROOT)
dbutils.jobs.taskValues.set(key="project_root", value=project_root)

# Load anything in layer_*_<color>/*.json
job_settings = {}
for color in [ "bronze", "silver", "gold" ]:
    paths = glob(f'./layer_*_{color}/*.json')
    job_settings[color] = [
        {"table": path.split("/")[-1].split(".")[0], "project_root": project_root}
        for path in paths]
for key, value in job_settings.items():
    dbutils.jobs.taskValues.set(key=key, value=value)

# Load anything in layer_*/*.json if it has a key "dst_table_name"
history_settings = []
for p in glob('./layer_*/*.json'):
    with open(p) as f:
        settings = json.load(f)
        settings = apply_job_type(settings)
        if str(settings.get("build_history", "false")).lower() == "true":
            entry = {"full_table_name": settings["dst_table_name"]}
            if "history_schema" in settings:
                entry["history_schema"] = settings["history_schema"]
            history_settings.append(entry)
dbutils.jobs.taskValues.set(key="history_settings", value=history_settings)

# Sanity check
validate_settings(project_root, dbutils)
initialize_empty_tables(project_root, spark)

# Save job configuration
save_job_configuration(dbutils, f"/Volumes/{catalog}/bronze/utility/jobs")